In [ ]:
import pandas as pd
import geopandas as gpd
import folium
from folium.features import GeoJsonTooltip
import branca.colormap as cm

# Đọc dữ liệu bất động sản từ file CSV
data = pd.read_csv('../../Data/cleanedData/cleaned_data.csv')

# Loại bỏ các hàng có giá trị thiếu trong cột 'Quận/Huyện' và 'Mức giá'
heatmap_data = data.dropna(subset=['Quận/Huyện', 'Mức giá'])

# Chuyển đổi cột 'Mức giá' thành kiểu số
heatmap_data['Mức giá'] = pd.to_numeric(heatmap_data['Mức giá'], errors='coerce')

# Tính giá trung bình theo quận/huyện
avg_price_by_district = heatmap_data.groupby('Quận/Huyện')['Mức giá'].mean().reset_index()

# Đọc shapefile của Việt Nam và lọc lấy Hà Nội
vietnam_map = gpd.read_file('../Map/HaNoiMap')
hanoi_map = vietnam_map[vietnam_map['NAME_1'] == 'Hà Nội']

# Ghép dữ liệu giá trung bình vào bản đồ Hà Nội
hanoi_map = hanoi_map.merge(avg_price_by_district, left_on='NAME_2', right_on='Quận/Huyện', how='left')

# Tọa độ trung tâm của Hà Nội
hanoi_center = [21.0285, 105.8542]

# Tạo một bản đồ nền bằng folium với nền mờ
m = folium.Map(
    location=hanoi_center,
    zoom_start=10,
    tiles="CartoDB Positron",  # Nền mờ
    max_bounds=True,          # Giữ bản đồ trong giới hạn
    dragging=False,           # Không cho phép kéo bản đồ
    zoom_control=False,       # Tắt nút thu phóng
    scrollWheelZoom=False     # Tắt thu phóng bằng cuộn chuột
)

# Tạo colormap
colormap = cm.linear.YlOrRd_09.scale(hanoi_map['Mức giá'].min(), hanoi_map['Mức giá'].max())

# Chuyển đổi GeoDataFrame thành GeoJson và thêm vào bản đồ
geojson = folium.GeoJson(
    hanoi_map,
    style_function=lambda feature: {
        'fillColor': colormap(feature['properties']['Mức giá']) if feature['properties']['Mức giá'] else 'lightblue',
        'color': 'black',
        'weight': 1,
        'fillOpacity': 0.6,
    },
    tooltip=GeoJsonTooltip(
        fields=['NAME_2', 'Mức giá'],
        aliases=['Quận/Huyện', 'Giá trung bình (triệu VND)'],
        localize=True
    ),
    highlight_function=lambda feature: {
        'fillColor': 'yellow',
        'color': 'black',
        'weight': 2,
        'fillOpacity': 0.7
    }
).add_to(m)

# Thêm colormap vào bản đồ
colormap.add_to(m)

# Lưu và hiển thị bản đồ
m


f:\anaconda\envs\BTL\Lib\site-packages\pyogrio\geopandas.py:265: UserWarning: More than one layer found in 'HaNoiMap': 'gadm41_VNM_2' (default), 'gadm41_VNM_3'. Specify layer parameter to avoid this warning.
  result = read_func(


In [ ]:
import pandas as pd
import geopandas as gpd
import folium
from folium.features import GeoJsonTooltip
import branca.colormap as cm

# Đọc dữ liệu bất động sản từ file CSV
data = pd.read_csv('../../Data/cleanedData/cleaned_data.csv')

# Loại bỏ các hàng có giá trị thiếu trong cột 'Xã/Phường' và 'Mức giá'
heatmap_data = data.dropna(subset=['Xã/Phường', 'Mức giá'])

# Chuyển đổi cột 'Mức giá' thành kiểu số
heatmap_data['Mức giá'] = pd.to_numeric(heatmap_data['Mức giá'], errors='coerce')

# Tính giá trung bình theo xã/phường
avg_price_by_ward = heatmap_data.groupby('Xã/Phường')['Mức giá'].mean().reset_index()

# Đọc shapefile của Việt Nam
vietnam_map = gpd.read_file('../Map/commune map')

def calculate_zoom(area_km2):
    """
    Xác định mức độ thu phóng (zoom level) dựa trên diện tích (km²).
    """
    if area_km2 > 500:
        return 10 # Quận/huyện lớn
    elif area_km2 > 100:
        return 12  # Quận/huyện trung bình
    elif area_km2 > 10:
        return 13  # Quận/huyện nhỏ
    else:
        return 14  # Quận/huyện rất nhỏ

def create_district_map(district_name):
    """
    Tạo bản đồ nhiệt cho quận/huyện được chọn, với zoom phù hợp dựa trên diện tích.
    """
    # Lọc bản đồ cho quận/huyện được chọn
    district_map = vietnam_map[(vietnam_map['NAME_1'] == 'Hà Nội') & (vietnam_map['NAME_2'] == district_name)]
    
    if district_map.empty:
        print(f"Không tìm thấy dữ liệu cho quận/huyện: {district_name}")
        return None
    
    # Tính diện tích (km²) của quận/huyện
    district_map = district_map.to_crs(epsg=3395)  # Chuyển hệ tọa độ để tính diện tích chính xác
    total_area_km2 = district_map['geometry'].area.sum() / 1e6  # Diện tích tổng cộng (m² -> km²)

    # Xác định mức độ thu phóng
    zoom_level = calculate_zoom(total_area_km2)

    # Ghép dữ liệu giá trung bình vào bản đồ
    district_map = district_map.to_crs(epsg=4326)  # Chuyển lại về hệ tọa độ địa lý
    district_map = district_map.merge(avg_price_by_ward, left_on='NAME_3', right_on='Xã/Phường', how='left')

    # Tọa độ trung tâm của quận/huyện
    district_center = [district_map.geometry.centroid.y.mean(), district_map.geometry.centroid.x.mean()]

    # Tạo bản đồ nền
    m_district = folium.Map(
        location=district_center,
        zoom_start=zoom_level,
        tiles="CartoDB Positron",  # Nền mờ
        max_bounds=True,
        dragging=True,  # Cho phép kéo bản đồ
        zoom_control=False,  # Bật nút thu phóng
        scrollWheelZoom=True  # Bật cuộn chuột để phóng to
    )

    # Tạo colormap
    colormap = cm.linear.YlOrRd_09.scale(
        district_map['Mức giá'].min(), 
        district_map['Mức giá'].max()
    )
    colormap.width = 100
    # Chuyển đổi GeoDataFrame thành GeoJson và thêm vào bản đồ
    folium.GeoJson(
        district_map,
        style_function=lambda feature: {
            'fillColor': colormap(feature['properties']['Mức giá']) if feature['properties']['Mức giá'] else 'lightblue',
            'color': 'black',
            'weight': 1,
            'fillOpacity': 0.6,
        },
        tooltip=GeoJsonTooltip(
            fields=['NAME_3', 'Mức giá'],
            aliases=['Xã/Phường', 'Giá trung bình (triệu VND)'],
            localize=True
        ),
        highlight_function=lambda feature: {
            'fillColor': 'yellow',
            'color': 'black',
            'weight': 2,
            'fillOpacity': 0.7
        }
    ).add_to(m_district)

    # Thêm colormap vào bản đồ
    colormap.add_to(m_district)

    return m_district

# Ví dụ: tạo bản đồ cho quận Chương Mỹ
district_name = "Hà Đông"  # Thay đổi tên quận/huyện tại đây
map_output = create_district_map(district_name)

# Hiển thị bản đồ
map_output


C:\Users\Admin\AppData\Local\Temp\ipykernel_5748\3573209837.py:58: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  district_center = [district_map.geometry.centroid.y.mean(), district_map.geometry.centroid.x.mean()]


In [45]:
import pandas as pd
import matplotlib.pyplot as plt

# Đường dẫn tệp
month_price_path = '../../Data/priceData/month_price.csv'
quarter_price_path = '../../Data/priceData/quarter_price.csv'
data_path = '../../Data/standardizedData/standardized_data.csv'

In [46]:
import pandas as pd
import plotly.graph_objs as go
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

# Đường dẫn tệp
month_price_path = '../../Data/priceData/month_price.csv'
quarter_price_path = '../../Data/priceData/quarter_price.csv'
data_path = '../../Data/standardizedData/standardized_data.csv'

# Đọc dữ liệu
data = pd.read_csv(data_path)
month_price = pd.read_csv(month_price_path)
quarter_price = pd.read_csv(quarter_price_path)

# Khởi tạo ứng dụng Dash
app = dash.Dash(__name__)

# Layout của ứng dụng
app.layout = html.Div([

    # Dropdown để chọn index
    dcc.Dropdown(
        id='index-dropdown',
        options=[{'label': f'Index {i}', 'value': i} for i in range(len(data))],
        value=15,  # Giá trị mặc định
         style={
        'width': '200px',
        'font-family': 'sans-serif',   # Kiểu chữ
        'font-size': '15px',                  # Kích thước chữ
        'border-radius': '10px',               # Bo tròn góc
        'background-color': '#CCCCCC',        # Màu nền
        'color': 'black',                      # Màu chữ
        
        }
    ),


    # Biểu đồ Plotly
    dcc.Graph(id='price-graph')
])

# Callback để cập nhật biểu đồ khi chọn index
@app.callback(
    Output('price-graph', 'figure'),
    [Input('index-dropdown', 'value')]
)
def update_graph(selected_index):
    code = data.loc[selected_index, 'Mã lịch sử giá']
    
    # Lấy dữ liệu giá tương ứng
    if code == 'M':
        price_data = month_price[month_price['index'] == selected_index].dropna(axis=1).drop(columns=['index'])
    elif code == 'Q':
        price_data = quarter_price[quarter_price['index'] == selected_index].dropna(axis=1).drop(columns=['index'])
    else:
        return go.Figure()

    # Tách dữ liệu giá thành các cột riêng biệt
    price_data = price_data.T
    price_data.columns = ['Giá']
    price_data[['Thấp nhất', 'Trung bình', 'Cao nhất']] = price_data['Giá'].str.split(expand=True)
    price_data.drop(columns=['Giá'], inplace=True)

    # Chuyển đổi kiểu dữ liệu sang float
    price_data = price_data.astype(float)

    # Tạo biểu đồ với Plotly
    fig = go.Figure()

    fig.add_trace(go.Scatter(x=price_data.index, y=price_data['Thấp nhất'], mode='lines', name='Thấp nhất', line=dict(color='#FFFF00')))
    fig.add_trace(go.Scatter(x=price_data.index, y=price_data['Trung bình'], mode='lines+markers', name='Trung bình', line=dict(color='#336666')))
    fig.add_trace(go.Scatter(x=price_data.index, y=price_data['Cao nhất'], mode='lines', name='Cao nhất', line=dict(color='#CC99FF')))
    # Tùy chỉnh biểu đồ
    fig.update_layout(
        title={
            'text': 'Biến động giá theo thời gian',  # Tiêu đề của biểu đồ
            'font': {
                'family': 'Arial, sans-serif',  # Kiểu chữ
                'size': 24,                    # Kích thước chữ
                'color': 'rgb(0, 123, 255)'     # Màu chữ
            },
            'x': 0.5,  # Căn giữa tiêu đề
            'xanchor': 'center'  # Căn giữa tiêu đề theo trục x
        },
        xaxis_title={
            'text': 'Thời gian',  # Tiêu đề của trục x
            'font': {
                'family': 'Arial, sans-serif',  # Kiểu chữ
                'size': 18,                    # Kích thước chữ
                'color': 'rgb(0, 123, 255)'     # Màu chữ
            }
        },
    
        xaxis_tickangle=-45,  # Góc nghiêng của các nhãn trục x
        legend_title={
            'text': 'Loại giá',  # Tiêu đề của legend
            'font': {
                'family': 'Arial, sans-serif',  # Kiểu chữ
                'size': 16,                    # Kích thước chữ
                'color': 'rgb(0, 123, 255)'     # Màu chữ
            }
        },
        template="plotly_white",  # Chọn template của biểu đồ
        hovermode='x unified'  # Thiết lập hovermode để tạo đường kẻ dọc tại điểm chuột
    )

    return fig


# Chạy ứng dụng
if __name__ == '__main__':
    app.run_server(debug=True)


In [47]:
# Kết hợp hai biểu đồ vào một trang HTML với liên kết đến các tệp gốc
with open("interactive_chart1.html", "r") as plotly_file, open("hanoi_map.html", "r") as folium_file:
    plotly_html = plotly_file.read()
    folium_html = folium_file.read()

combined_html = f"""
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <title>Kết hợp biểu đồ</title>
</head>
<body>

<h1><a href="interactive_chart1.html" target="_blank">Biểu đồ Biến động giá</a></h1>
<iframe srcdoc='{plotly_html}' width="100%" height="600"></iframe>

<h1><a href="hanoi_map.html" target="_blank">Bản đồ giá bất động sản Hà Nội</a></h1>
<iframe srcdoc='{folium_html}' width="100%" height="600"></iframe>

</body>
</html>
"""

with open("combined_charts.html", "w") as combined_file:
    combined_file.write(combined_html)


In [48]:
import pandas as pd
import plotly.graph_objs as go
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

# Đọc dữ liệu từ tệp CSV
data_path = '../../Data/priceData/average_prices.csv'
data = pd.read_csv(data_path)

# Lấy danh sách các quận/huyện duy nhất
districts = data['Quận/Huyện'].unique()

# Khởi tạo ứng dụng Dash với tên mới
app1 = dash.Dash(__name__, title="Biểu đồ Biến động Giá theo Quận/Huyện")

# Layout của ứng dụng với tên biểu đồ mới
app1.layout = html.Div([

    # Dropdown để chọn quận/huyện
    dcc.Dropdown(
        id='district-dropdown',
        options=[{'label': district, 'value': district} for district in districts],
        value=districts[0],  # Giá trị mặc định là quận/huyện đầu tiên
        style={
            'width': '200px',
            'font-family': 'sans-serif',
            'font-size': '15px',
            'border-radius': '10px',
            'background-color': '#CCCCCC',
            'color': 'black',
        }
    ),

    # Biểu đồ Plotly cho quận/huyện (đổi ID biểu đồ)
    dcc.Graph(id='district-price-graph')  # Đổi ID biểu đồ
])

# Callback để cập nhật biểu đồ khi chọn quận/huyện
@app1.callback(
    Output('district-price-graph', 'figure'),  # Đổi ID biểu đồ tại đây
    [Input('district-dropdown', 'value')]
)
def update_graph(selected_district):
    # Lọc dữ liệu theo quận/huyện được chọn
    district_data = data[data['Quận/Huyện'] == selected_district]

    # Tạo biểu đồ với Plotly
    fig1 = go.Figure()

    fig1.add_trace(go.Scatter(
        x=district_data['Thời gian'],
        y=district_data['Giá'],
        mode='lines+markers',
        name=f'Giá - {selected_district}',
        line=dict(color='#336666')
    ))

    # Tùy chỉnh biểu đồ
    fig1.update_layout(
        title={
            'text': f'Biến động giá theo thời gian tại {selected_district}',
            'font': {
                'family': 'Arial, sans-serif',
                'size': 24,
                'color': 'rgb(0, 123, 255)'
            },
            'x': 0.5,
            'xanchor': 'center'
        },
        xaxis_title='Thời gian',
        yaxis_title='Giá (triệu/m²)',
        xaxis_tickangle=-45,
        legend_title='Loại giá',
        template="plotly_white",
        hovermode='x unified'
    )

    return fig1

# Chạy ứng dụng
if __name__ == '__main__':
    app1.run_server(debug=True)


In [49]:
import pandas as pd
import plotly.graph_objs as go
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import re

# Đọc dữ liệu từ tệp CSV
data_path = '../../Data/cleanedData/cleaned_data_project.csv'  # Đường dẫn đến tệp dữ liệu
data = pd.read_csv(data_path)

# Hàm xử lý dữ liệu 'Lịch sử giá'
def parse_price_history(row):
    """Chuyển đổi chuỗi 'Lịch sử giá' thành DataFrame với thời gian và giá."""
    entries = row.split(";")
    time, price = [], []
    for entry in entries:
        if ":" in entry:
            t, p = entry.split(":")
            time.append(t.strip())
            # Làm sạch giá trị giá bằng regex
            cleaned_price = re.sub(r"[^\d.]", "", p)  # Loại bỏ ký tự không phải số hoặc dấu chấm
            if cleaned_price:  # Chỉ thêm nếu giá trị không rỗng
                price.append(float(cleaned_price))
    return pd.DataFrame({"Thời gian": time, "Giá": price})

# Tạo dictionary chứa dữ liệu lịch sử giá cho từng dự án
project_price_data = {
    project: parse_price_history(history)
    for project, history in zip(data['Tên dự án'], data['Lịch sử giá'])
}

# Lấy danh sách các dự án duy nhất
projects = data['Tên dự án'].unique()

# Khởi tạo ứng dụng Dash
app = dash.Dash(__name__, title="Biểu đồ Biến động Giá theo Dự án")

# Layout của ứng dụng
app.layout = html.Div([
    # Dropdown để chọn dự án
    dcc.Dropdown(
        id='project-dropdown',
        options=[{'label': project, 'value': project} for project in projects],
        value=projects[0],  # Giá trị mặc định là dự án đầu tiên
        style={
            'width': '400px',
            'font-family': 'Arial, sans-serif',
            'font-size': '15px',
            'border-radius': '10px',
            'background-color': '#f0f0f0',
            'color': '#000',
            'margin-bottom': '20px',
        }
    ),

    # Biểu đồ Plotly
    dcc.Graph(id='project-price-graph')
])

# Callback để cập nhật biểu đồ khi chọn dự án
@app.callback(
    Output('project-price-graph', 'figure'),
    [Input('project-dropdown', 'value')]
)
def update_graph(selected_project):
    # Lấy dữ liệu lịch sử giá của dự án được chọn
    project_data = project_price_data[selected_project]

    # Tạo biểu đồ với Plotly
    fig = go.Figure()

    fig.add_trace(go.Scatter(
        x=project_data['Thời gian'],
        y=project_data['Giá'],
        mode='lines+markers',
        name=f'Giá - {selected_project}',
        line=dict(color='#336699', width=2)
    ))

    # Tùy chỉnh biểu đồ
    fig.update_layout(
        title={
            'text': f'Biến động giá theo thời gian của dự án: {selected_project}',
            'font': {
                'family': 'Arial, sans-serif',
                'size': 24,
                'color': '#007bff'
            },
            'x': 0.5,
            'xanchor': 'center'
        },
        xaxis_title='Thời gian',
        yaxis_title='Giá (triệu/m²)',
        xaxis_tickangle=-45,
        legend_title='Loại giá',
        template="plotly_white",
        hovermode='x unified'
    )

    return fig

# Chạy ứng dụng
if __name__ == '__main__':
    app.run_server(debug=True)


In [47]:
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html

# Đọc dữ liệu từ tệp CSV
data_path = '../../Data/originalData/data_project.csv'  # Đường dẫn đến tệp dữ liệu
data = pd.read_csv(data_path)

# Đếm số lượng dự án đầu tư vào từng quận/huyện
district_project_counts = data['Quận/Huyện'].value_counts().reset_index()
district_project_counts.columns = ['Quận/Huyện', 'Số lượng dự án']

# Khởi tạo ứng dụng Dash
app = dash.Dash(__name__, title="Số lượng dự án theo Quận/Huyện")

# Layout của ứng dụng
app.layout = html.Div([
    html.H1("Số lượng dự án đầu tư theo Quận/Huyện", style={'text-align': 'center'}),

    # Biểu đồ cột
    dcc.Graph(
        id='district-project-bar-chart',
        figure=px.bar(
            district_project_counts,
            x='Quận/Huyện',
            y='Số lượng dự án',
            text='Số lượng dự án',
            title="Số lượng dự án theo Quận/Huyện",
            labels={'Quận/Huyện': 'Quận/Huyện', 'Số lượng dự án': 'Số lượng dự án'},
            color='Số lượng dự án',
            color_continuous_scale=px.colors.sequential.Viridis
        ).update_traces(textposition='outside')  # Hiển thị số lượng dự án trên đầu mỗi cột
        .update_layout(
            title={
                'font': {'size': 20, 'family': 'Arial, sans-serif', 'color': '#333'},
                'x': 0.5,
                'xanchor': 'center'
            },
            height=520,
            xaxis_tickangle=-45,
            template='plotly_white'
        )
    )
])

# Chạy ứng dụng
if __name__ == '__main__':
    app.run_server(debug=True)


In [48]:
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

# Đọc dữ liệu từ tệp CSV
data_path = '../../Data/cleanedData/cleaned_data.csv'  # Đường dẫn đến tệp dữ liệu
data = pd.read_csv(data_path)

# Làm sạch và chuẩn bị dữ liệu: Loại bỏ các dòng có giá trị 'Không có dữ liệu' trong cột "Mức giá (triệu/m²)"
data = data[data['Mức giá (triệu/m²)'].notnull()]

# Khởi tạo ứng dụng Dash
app = dash.Dash(__name__, title="Biểu đồ Giá trung bình theo Xã/Phường")

# Lọc quận/huyện có ít nhất 5 xã/phường
district_counts = data.groupby('Quận/Huyện')['Xã/Phường'].nunique()  # Đếm số xã/phường trong mỗi quận/huyện
valid_districts = district_counts[district_counts >= 5].index  # Lọc quận/huyện có >= 5 xã/phường

# Lấy danh sách các quận/huyện đủ điều kiện
districts = valid_districts.tolist()

# Layout của ứng dụng
app.layout = html.Div([
    html.H1("Biểu đồ giá trung bình theo Xã/Phường trong Quận/Huyện", style={'text-align': 'center'}),

    # Dropdown để chọn quận/huyện
    dcc.Dropdown(
        id='district-dropdown',
        options=[{'label': district, 'value': district} for district in districts],
        value=districts[0],  # Giá trị mặc định là quận/huyện đầu tiên
        style={'width': '50%', 'margin': 'auto', 'font-size': '16px'}
    ),

    # Biểu đồ cột hiển thị giá trung bình theo xã/phường
    dcc.Graph(id='price-bar-chart')
])

# Callback để cập nhật biểu đồ khi chọn quận/huyện
@app.callback(
    Output('price-bar-chart', 'figure'),
    [Input('district-dropdown', 'value')]
)
def update_graph(selected_district):
    # Lọc dữ liệu theo quận/huyện được chọn
    district_data = data[data['Quận/Huyện'] == selected_district]

    # Tính giá trung bình cho từng xã/phường
    avg_price_per_district = district_data.groupby('Xã/Phường')['Mức giá (triệu/m²)'].mean().reset_index()

    # Sắp xếp giá trung bình từ cao xuống thấp
    avg_price_per_district = avg_price_per_district.sort_values(by='Mức giá (triệu/m²)', ascending=False)

    # Tạo biểu đồ cột
    fig = px.bar(
        avg_price_per_district,
        x='Xã/Phường',  # Trục x là xã/phường
        y='Mức giá (triệu/m²)',  # Trục y là mức giá trung bình
        title=f'Giá trung bình theo Xã/Phường tại {selected_district}',
        labels={'Xã/Phường': 'Xã/Phường', 'Mức giá (triệu/m²)': 'Giá trung bình (triệu/m²)'},
        color='Mức giá (triệu/m²)',  # Màu sắc theo mức giá
        color_continuous_scale=px.colors.sequential.Viridis
    )

    # Cố định độ rộng các cột
    # Chúng ta có thể điều chỉnh width của cột thông qua 'bargap' và 'bargroupgap'
    fig.update_layout(
        xaxis_tickangle=-45,  # Quay nhãn trục x cho dễ đọc
        title={
            'font': {'size': 20, 'family': 'Arial, sans-serif', 'color': '#333'},
            'x': 0.5,
            'xanchor': 'center'
        },
        height=500,
        bargap=0.3,  # Khoảng cách giữa các cột
        bargroupgap=0.4,  # Khoảng cách nhóm cột
        template='plotly_white'
    )

    return fig

# Chạy ứng dụng
if __name__ == '__main__':
    app.run_server(debug=True)


In [49]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

# Đọc dữ liệu từ tệp CSV
data_path = '../../Data/cleanedData/cleaned_data.csv'  # Đường dẫn đến tệp dữ liệu
data = pd.read_csv(data_path)

# Làm sạch và chuẩn bị dữ liệu: Loại bỏ các dòng có giá trị 'Không có dữ liệu' trong cột số
data = data.select_dtypes(include=['float64', 'int64'])  # Lọc chỉ các cột số

# Tính toán độ tương quan giữa các cột
correlation_matrix = data.corr()

# Chuyển đổi ma trận tương quan thành dạng dài (long format) để dễ vẽ biểu đồ
corr_long = correlation_matrix.unstack().reset_index()
corr_long.columns = ['Trường 1', 'Trường 2', 'Độ tương quan']

# Lọc bỏ các cặp trường trùng lặp và có độ tương quan bằng 1
corr_long = corr_long[corr_long['Trường 1'] != corr_long['Trường 2']]
corr_long = corr_long[corr_long['Độ tương quan'].abs() > 0.5]  # Chỉ giữ lại các cặp có độ tương quan > 0.5

# Tạo biểu đồ cột
fig = px.bar(
    corr_long,
    x='Trường 1', 
    y='Độ tương quan',
    color='Trường 2',
    title="Độ tương quan giữa các cặp trường",
    labels={'Trường 1': 'Trường 1', 'Trường 2': 'Trường 2', 'Độ tương quan': 'Độ tương quan'},
    color_continuous_scale=px.colors.sequential.Viridis
)

# Cải thiện hiển thị
fig.update_layout(
    xaxis_tickangle=45,  # Quay nhãn trục x cho dễ đọc
    title={
        'font': {'size': 20, 'family': 'Arial, sans-serif', 'color': '#333'},
        'x': 0.5,
        'xanchor': 'center'
    },
    height=600,
    template='plotly_white'
)

# Hiển thị biểu đồ
fig.show()


In [50]:
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html
from dash.dependencies import Input, Output

# Đọc dữ liệu từ tệp CSV
data_path = '../../Data/cleanedData/cleaned_data.csv'  # Đường dẫn đến tệp dữ liệu
data = pd.read_csv(data_path)

# Làm sạch và chuẩn bị dữ liệu: Loại bỏ các dòng có giá trị 'Không có dữ liệu' trong cột "Mức giá (triệu/m²)"
data = data[data['Mức giá (triệu/m²)'].notnull()]

# Khởi tạo ứng dụng Dash
app = dash.Dash(__name__, title="Biểu đồ giá trung bình của các Chủ đầu tư trong Quận/Huyện")

# Lấy danh sách các quận/huyện duy nhất
districts = data['Quận/Huyện'].unique()

# Layout của ứng dụng
app.layout = html.Div([
    html.H1("Biểu đồ giá trung bình của các Chủ đầu tư trong Quận/Huyện", style={'text-align': 'center'}),

    # Dropdown để chọn quận/huyện
    dcc.Dropdown(
        id='district-dropdown',
        options=[{'label': district, 'value': district} for district in districts],
        value=districts[0],  # Giá trị mặc định là quận/huyện đầu tiên
        style={'width': '50%', 'margin': 'auto', 'font-size': '16px'}
    ),

    # Biểu đồ cột hiển thị giá trung bình của các Chủ đầu tư
    dcc.Graph(id='price-bar-chart')
])

# Callback để cập nhật biểu đồ khi chọn quận/huyện
@app.callback(
    Output('price-bar-chart', 'figure'),
    [Input('district-dropdown', 'value')]
)
def update_graph(selected_district):
    # Lọc dữ liệu theo quận/huyện được chọn
    district_data = data[data['Quận/Huyện'] == selected_district]

    # Tính giá trung bình cho từng Chủ đầu tư
    avg_price_per_investor = district_data.groupby('Chủ đầu tư')['Mức giá (triệu/m²)'].mean().reset_index()

    # Tạo biểu đồ cột
    fig = px.bar(
        avg_price_per_investor,
        x='Chủ đầu tư',  # Trục x là Chủ đầu tư
        y='Mức giá (triệu/m²)',  # Trục y là mức giá trung bình
        title=f'Giá trung bình của các Chủ đầu tư tại {selected_district}',
        labels={'Chủ đầu tư': 'Chủ đầu tư', 'Mức giá (triệu/m²)': 'Giá trung bình (triệu/m²)'},
        color='Mức giá (triệu/m²)',  # Màu sắc theo mức giá
        color_continuous_scale=px.colors.sequential.Viridis
    )

    # Cải thiện hiển thị: Tùy chỉnh độ rộng cột khi số lượng cột ít
    num_investors = len(avg_price_per_investor)
    bargap = 0.5 if num_investors <= 5 else 0.3  # Nếu ít cột thì giảm khoảng cách giữa các cột
    bargroupgap = 0.3

    fig.update_layout(
        xaxis_tickangle=-45,  # Quay nhãn trục x cho dễ đọc
        title={
            'font': {'size': 20, 'family': 'Arial, sans-serif', 'color': '#333'},
            'x': 0.5,
            'xanchor': 'center'
        },
        height=600,
        bargap=bargap,  # Khoảng cách giữa các cột
        bargroupgap=bargroupgap,  # Khoảng cách nhóm cột
        template='plotly_white'
    )

    return fig

# Chạy ứng dụng
if __name__ == '__main__':
    app.run_server(debug=True)
